# Network Modeling with SEIRS+ Notebook 

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson
from seirsplus.models import *
from network_utils import *
from stats_utils import *
from intervention_utils import *
from camp_params import *
import pickle as pkl

### 1) Load base graph for Moria

In [ ]:
graph, nodes_per_struct = load_graph("Moria_wNeighbors")

In [ ]:
graph, nodes_per_struct[:10]

#### 1.1) We will add a general food queue which represents the current state of the camp

In [ ]:
food_weight = 0.407
graph_1fq = connect_food_queue(graph, nodes_per_struct, food_weight, "food")

### 2) Plot the basic network degrees

In [ ]:
# Degrees without food queue, just neighbors
min_G, max_G = min_degree(graph), max_degree(graph)
print("Min degree:", min_G, "Max degree:", max_G)
plot_degree_distn(graph, max_degree=max_G)

In [ ]:
# Degrees with 1 food queue
min_G, max_G = min_degree(graph_1fq), max_degree(graph_1fq)
print("Min degree:", min_G, "Max degree:", max_G)
plot_degree_distn(graph_1fq, max_degree=max_G)

### 3) Create node groups of 10 year age bucket to track the results

In [ ]:
node_groups=create_node_groups(graph)

### 4) Define SEIRS model parameters

In [ ]:
transmission_rate = 1.28
progression_rate = round(1/5.1, 3)
recovery_rate = 0.056 # Approx 1/18 -> Recovery occurs after 18 days
hosp_rate = round(1/11.4, 3) #1/6.3 # From Tucker Model
# crit_rate = 0.3 # From camp_params
crit_rate = list((sample_pop["death_rate"] / sample_pop["prob_symptomatic"]) / sample_pop["prob_hospitalisation"])
death_rate = 0.75

prob_global_contact = 1
prob_detected_global_contact = 1

# prob_hosp_to_critical = list(sample_pop["death_rate"]/sample_pop["prob_hospitalisation"])
prob_death = list(sample_pop["death_rate"])
prob_asymptomatic = list(1 - sample_pop["prob_symptomatic"])
prob_symp_to_hosp = list(sample_pop["prob_hospitalisation"])

init_symp_cases = 1
init_asymp_cases = 0

### 5) Running the base model

In [ ]:
# Model construction
ref_model = SymptomaticSEIRSNetworkModel(G=graph_1fq, beta=transmission_rate, sigma=progression_rate, gamma=recovery_rate, 
                                         lamda=progression_rate, mu_H=crit_rate, eta=hosp_rate, p=prob_global_contact, a=prob_asymptomatic, f=death_rate, 
                                         h=prob_symp_to_hosp, q=prob_detected_global_contact, initI_S=init_symp_cases, initI_A=init_asymp_cases, store_Xseries=True)

In [ ]:
t_steps = 200
node_states, simulation_results = run_simulation(ref_model, t_steps)

In [ ]:
# Model name for storage
fig_name = f"BaseSympModel_HW={household_weight}_NW={neighbor_weight}_FW={food_weight}_TransR={transmission_rate}_RecR={recovery_rate}_ProgR={progression_rate}_HospR={hosp_rate}_CritR={sum(crit_rate)/len(crit_rate)}_DeathR={death_rate}_initI_S={init_symp_cases}_initI_A={init_asymp_cases}_T={t_steps}"

In [ ]:
#here we set up a parallel computing to run experiments in parallel and aggregate the results

In [ ]:
output_df = results_to_df(simulation_results, store=True, store_name=f"results/{fig_name}.csv")

In [ ]:
output_df

In [ ]:
fig, ax = ref_model.figure_basic()#vlines=interventions.get_checkpoints()['t'])
fig.savefig(f"plots/{fig_name}_figBasic.png")

### 6) Define Interventions

With the interventions module, we can create an intervention with just a time step and a custom network referring to that intervention, as well as remove/edit them from the list. The method get_checkpoints() will allow us to get the dictionary to be fed to the SEIRS+ model

#### 6.1) First, define the intervention graphs

In [ ]:
# This is social distancing within the food queue 
distancing_graph = remove_edges_from_graph(graph_1fq, scale=10, edge_label_list=["food"], min_num_edges=4)

# This also includes neighbor/friendship edges
quarantine_graph = remove_edges_from_graph(graph_1fq, scale=2, edge_label_list=["food", "friendship"], min_num_edges=2)

# Create graphs with multiple sectoring of food queues - in this case, 1 and 2 queues per each block (4 and 8 food queues respectively)
graph_4fq = create_multiple_food_queues(graph, 1, food_weight, nodes_per_struct, [grid_isoboxes, grid_block1, grid_block2, grid_block3])
graph_8fq = create_multiple_food_queues(graph, 2, food_weight, nodes_per_struct, [grid_isoboxes, grid_block1, grid_block2, grid_block3])

# Create quarantine graphs for the sectored graphs
quarantine_graph_4fq = remove_edges_from_graph(graph_4fq, scale=2, edge_label_list=[f"food_{i}" for i in range(4)] + ["friendship"], min_num_edges=2)
quarantine_graph_8fq = remove_edges_from_graph(graph_8fq, scale=2, edge_label_list=[f"food_{i}" for i in range(8)] + ["friendship"], min_num_edges=2)

In [ ]:
min_D, max_D = min_degree(distancing_graph), max_degree(distancing_graph)
print("Min degree:", min_D, "Max degree:", max_D)

# As we can see, there isn't much of a difference when we only cut food-queue interactions, 
# which means we need a harder distancing policy
plot_degree_distn(distancing_graph, max_degree=max_D)

In [ ]:
min_Q, max_Q = min_degree(quarantine_graph), max_degree(quarantine_graph)
print("Min degree:", min_Q, "Max degree:", max_Q)

plot_degree_distn(quarantine_graph, max_degree=max_Q)

In [ ]:
# 4 food queues 
min_4fq, max_4fq = min_degree(graph_4fq), max_degree(graph_4fq)
print("Min degree:", min_4fq, "Max degree:", max_4fq)

plot_degree_distn(quarantine_graph, max_degree=max_4fq)

In [ ]:
# 8 food queues
min_8fq, max_8fq = min_degree(graph_8fq), max_degree(graph_8fq)
print("Min degree:", min_8fq, "Max degree:", max_8fq)

plot_degree_distn(quarantine_graph, max_degree=max_8fq)

#### 6.2) Construct the checkpoints

In [ ]:
interventions = Interventions()
reduction_percentage = 0.5

# Simulate quarantine + masks
interventions.add(quarantine_graph, 3, beta=transmission_rate*reduction_percentage)

# Simulate HALT of quarantine but people still have to wear masks
interventions.add(graph_1fq, 63, beta=transmission_rate*reduction_percentage)

# Simulate HALT of wearing masks
interventions.add(graph_1fq, 93, beta=transmission_rate)

checkpoints = interventions.get_checkpoints()

### 7) Running the model with interventions

In [ ]:
# Model construction with interventions on the model with 1 food queue
interventions_model = SymptomaticSEIRSNetworkModel(G=graph_1fq, Q=quarantine_graph, beta=transmission_rate, sigma=progression_rate, gamma=recovery_rate, 
                                         lamda=progression_rate, mu_H=crit_rate, eta=hosp_rate, p=prob_global_contact, a=prob_asymptomatic, f=death_rate, 
                                         h=prob_symp_to_hosp, q=prob_detected_global_contact, initI_S=init_symp_cases, initI_A=init_asymp_cases, store_Xseries=True)

In [ ]:
t_steps = 200
intervention_node_states, intervention_results = run_simulation(interventions_model, t_steps, checkpoints)

In [ ]:
# Model name for storage - adjust accordingly to interventions
mean_degree = round(sum([degree for node, degree in quarantine_graph.degree])/len(quarantine_graph.degree), 3) # Get the mean degree
quarantine_duration = "3-63"

fig_name = f"InterventionsSympModel_HW={household_weight}_NW={neighbor_weight}_FW={food_weight}_TransR={transmission_rate}_RecR={recovery_rate}_ProgR={progression_rate}_HospR={hosp_rate}_CritRate={sum(crit_rate)/len(crit_rate)}_DeathR={death_rate}_initI_S={init_symp_cases}_initI_A={init_asymp_cases}_T={t_steps}_QDeg={mean_degree}_QDur={quarantine_duration}"

In [ ]:
output_df = results_to_df(intervention_results, store=True, store_name=f"results/{fig_name}.csv")

In [ ]:
output_df

In [ ]:
fig, ax = interventions_model.figure_basic()#vlines=interventions.get_checkpoints()['t'])
fig.savefig(f"plots/{fig_name}_figBasic.png")

#### 7.1) Running the base model with multiple food queues

In [ ]:
# Model construction for multiple food queues but no more interventions
model_4fq = SymptomaticSEIRSNetworkModel(G=graph_4fq, beta=transmission_rate, sigma=progression_rate, gamma=recovery_rate, 
                                         lamda=progression_rate, mu_H=crit_rate, eta=hosp_rate, p=prob_global_contact, a=prob_asymptomatic, f=death_rate, 
                                         h=prob_symp_to_hosp, q=prob_detected_global_contact, initI_S=init_symp_cases, initI_A=init_asymp_cases, store_Xseries=True)

model_8fq = SymptomaticSEIRSNetworkModel(G=graph_8fq, beta=transmission_rate, sigma=progression_rate, gamma=recovery_rate, 
                                         lamda=progression_rate, mu_H=crit_rate, eta=hosp_rate, p=prob_global_contact, a=prob_asymptomatic, f=death_rate, 
                                         h=prob_symp_to_hosp, q=prob_detected_global_contact, initI_S=init_symp_cases, initI_A=init_asymp_cases, store_Xseries=True)

In [ ]:
t_steps = 200
node_states, simulation_results_4fq = run_simulation(model_4fq, t_steps)

In [ ]:
# Model name for storage
fig_name = f"SympModel_4FQ_HW={household_weight}_NW={neighbor_weight}_FW={food_weight}_TransR={transmission_rate}_RecR={recovery_rate}_ProgR={progression_rate}_HospR={hosp_rate}_CritR={sum(crit_rate)/len(crit_rate)}_DeathR={death_rate}_initI_S={init_symp_cases}_initI_A={init_asymp_cases}_T={t_steps}"

In [ ]:
output_df = results_to_df(simulation_results_4fq, store=True, store_name=f"results/{fig_name}.csv")

In [ ]:
output_df

In [ ]:
fig, ax = model_4fq.figure_basic()#vlines=interventions.get_checkpoints()['t'])
fig.savefig(f"plots/{fig_name}_figBasic.png")

In [ ]:
t_steps = 200
node_states, simulation_results_8fq = run_simulation(model_8fq t_steps)

In [ ]:
# Model name for storage
fig_name = f"SympModel_8FQ_HW={household_weight}_NW={neighbor_weight}_FW={food_weight}_TransR={transmission_rate}_RecR={recovery_rate}_ProgR={progression_rate}_HospR={hosp_rate}_CritR={sum(crit_rate)/len(crit_rate)}_DeathR={death_rate}_initI_S={init_symp_cases}_initI_A={init_asymp_cases}_T={t_steps}"

In [ ]:
output_df = results_to_df(simulation_results_8fq, store=True, store_name=f"results/{fig_name}.csv")

In [ ]:
output_df

In [ ]:
fig, ax = model_8fq.figure_basic()#vlines=interventions.get_checkpoints()['t'])
fig.savefig(f"plots/{fig_name}_figBasic.png")

#### 7.2) Running the multiple food queues model with interventions

In [ ]:
# Model construction for multiple food queues in addition to other interventions
interventions_model_4fq = SymptomaticSEIRSNetworkModel(G=graph_4fq, Q=quarantine_graph_4fq beta=transmission_rate, sigma=progression_rate, gamma=recovery_rate, 
                                         lamda=progression_rate, mu_H=crit_rate, eta=hosp_rate, p=prob_global_contact, a=prob_asymptomatic, f=death_rate, 
                                         h=prob_symp_to_hosp, q=prob_detected_global_contact, initI_S=init_symp_cases, initI_A=init_asymp_cases, store_Xseries=True)

interventions_model_8fq = SymptomaticSEIRSNetworkModel(G=graph_8fq, Q=quarantine_graph_8fq, beta=transmission_rate, sigma=progression_rate, gamma=recovery_rate, 
                                         lamda=progression_rate, mu_H=crit_rate, eta=hosp_rate, p=prob_global_contact, a=prob_asymptomatic, f=death_rate, 
                                         h=prob_symp_to_hosp, q=prob_detected_global_contact, initI_S=init_symp_cases, initI_A=init_asymp_cases, store_Xseries=True)

In [ ]:
## Interventions for graph with 4 food queues 
interventions.clear()

# Simulate quarantine + masks
interventions.add(quarantine_graph_4fq, 3, beta=transmission_rate*reduction_percentage)

# Simulate HALT of quarantine but people still have to wear masks
interventions.add(graph_4fq, 63, beta=transmission_rate*reduction_percentage)

# Simulate HALT of wearing masks
interventions.add(graph_4fq, 93, beta=transmission_rate)

checkpoints = interventions.get_checkpoints()

In [ ]:
t_steps = 200
node_states, simulation_results_4fq = run_simulation(model_4fq, t_steps)

In [ ]:
# Model name for storage - adjust accordingly to interventions
mean_degree = round(sum([degree for node, degree in quarantine_graph_4fq.degree])/len(quarantine_graph_4fq.degree), 3) # Get the mean degree
quarantine_duration = "3-63"

fig_name = f"InterventionsSympModel4FQ_HW={household_weight}_NW={neighbor_weight}_FW={food_weight}_TransR={transmission_rate}_RecR={recovery_rate}_ProgR={progression_rate}_HospR={hosp_rate}_CritRate={sum(crit_rate)/len(crit_rate)}_DeathR={death_rate}_initI_S={init_symp_cases}_initI_A={init_asymp_cases}_T={t_steps}_QDeg={mean_degree}_QDur={quarantine_duration}"

In [ ]:
output_df = results_to_df(intervention_results, store=True, store_name=f"results/{fig_name}.csv")

In [ ]:
output_df

In [ ]:
fig, ax = interventions_model_4fq.figure_basic()#vlines=interventions.get_checkpoints()['t'])
fig.savefig(f"plots/{fig_name}_figBasic.png")

In [ ]:
## Interventions for graph with 8 food queues 
interventions.clear()

# Simulate quarantine + masks
interventions.add(quarantine_graph_8fq, 3, beta=transmission_rate*reduction_percentage)

# Simulate HALT of quarantine but people still have to wear masks
interventions.add(graph_8fq, 63, beta=transmission_rate*reduction_percentage)

# Simulate HALT of wearing masks
interventions.add(graph_8fq, 93, beta=transmission_rate)

checkpoints = interventions.get_checkpoints()

In [ ]:
t_steps = 200
node_states, simulation_results_4fq = run_simulation(model_4fq, t_steps)

In [ ]:
# Model name for storage - adjust accordingly to interventions
mean_degree = round(sum([degree for node, degree in quarantine_graph_8fq.degree])/len(quarantine_graph_8fq.degree), 3) # Get the mean degree
quarantine_duration = "3-63"

fig_name = f"InterventionsSympModel8FQ_HW={household_weight}_NW={neighbor_weight}_FW={food_weight}_TransR={transmission_rate}_RecR={recovery_rate}_ProgR={progression_rate}_HospR={hosp_rate}_CritRate={sum(crit_rate)/len(crit_rate)}_DeathR={death_rate}_initI_S={init_symp_cases}_initI_A={init_asymp_cases}_T={t_steps}_QDeg={mean_degree}_QDur={quarantine_duration}"

In [ ]:
output_df = results_to_df(intervention_results, store=True, store_name=f"results/{fig_name}.csv")

In [ ]:
output_df

In [ ]:
fig, ax = interventions_model_8fq.figure_basic()#vlines=interventions.get_checkpoints()['t'])
fig.savefig(f"plots/{fig_name}_figBasic.png")